In [1]:
#import pandas and data
import pandas as pd
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [2]:
#convert to datetime
prices["updated_time"] = pd.to_datetime(prices["updated_at"])
sales["ordered_time"] = pd.to_datetime(sales["ordered_at"])


In [3]:
#create original_prices table
product_mintime1 = prices.groupby('product_id')['updated_time'].min()
product_mintime = product_mintime1.to_frame()
prices_mintime = product_mintime.merge(prices, on=['product_id', 'updated_time'])
prices_mintime['updated_time_1800'] = pd.to_datetime('2000-09-10 16:37:00')
original_prices = prices_mintime[['product_id', 'old_price', 'updated_time_1800']]
#rename columns' name of original_prices table
original_prices.columns = ['product_id', 'new_price', 'updated_time']


In [4]:
#concat prices and original_prices table => full_prices
full_prices = pd.concat([prices[['product_id', 'new_price', 'updated_time']], original_prices])


In [5]:
#sort values in full_prices and sales tables to use asof
s_full_prices = full_prices.sort_values('updated_time')
s_sales = sales.sort_values('ordered_time')
#create sales_prices by joining sales and prices data
sales_prices = pd.merge_asof(s_sales, s_full_prices, left_on='ordered_time', right_on='updated_time', by="product_id")

In [6]:
#sum revenue for each product at each price
sales_prices['revenue'] = sales_prices['quantity_ordered'] * sales_prices['new_price']
revenue = sales_prices.groupby(['product_id', 'new_price'])['revenue'].sum()


In [7]:
print(revenue)

product_id  new_price
64          239000        956000
3954203     57500          57500
            60000         180000
            64000         640000
3998909     15500          15500
            16500         231000
            17000          34000
4085861     52000        1040000
            53500        2140000
            58000        2204000
            60000         180000
            62500        1812500
            67000         871000
Name: revenue, dtype: int64
